# Casificador de imagenes entrenando yolov8 con datos propios

En este notebook pensado para ejecutarse en google colab, se entrena un modelo de clasificación de imagenes con yolov8. El modelo se entrena con las imagenes de comidas porporcionadas para la realización del concurso de Oracle en nuwe.io. El modelo aprende a diferenciar 8 diferentes clases (de la 0 a la 7), donde cada número representa una comida diferente.

Para conseguirlo utilizaremos la función de entrenamiento incluida en la CLI de yolov8, realizando transfer learning con el modelo preentrenado de yolov8.

Primero comprobamos que el entorno nos ha proporcionado una GPU para poder entrenar el modelo. Si no es así, podemos cambiar el entorno de ejecución a uno que nos proporcione una GPU.

In [ ]:
!nvidia-smi

Sun Mar 19 19:46:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Vamos a configurar ngrok para poder monitorizar el proceso de entrenamiento desde nuestro ordenador. Para ello, primero nos descargamos el paquete correspondiente de python.

In [21]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19790 sha256=8d7962274a3f4884acdd61f77de92cc9d74fba4f9e0c310fc963f6ad041d87bc
  Stored in directory: /root/.cache/pip/wheels/f6/89/59/49d4249e00957e94813ac136a335d10ed2e09a856c5096f95c
Successfully built pyngrok


Conectamos ngrok con nuestra cuenta (utilizando el token que nos proporciona ngrok al registrarnos), y abrimos el puerto 6006 para poder acceder a él desde la ip que nos proporcionan.

In [24]:
from pyngrok import ngrok
ngrok.set_auth_token("<your authtoken>")
public_url = ngrok.connect(6006).public_url
print(public_url)

http://59c8-34-147-5-164.ngrok.io


Montamos nuestro drive para poder acceder a los datos que vamos a utilizar.

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


Definimos nuestra ruta de trabajo

In [3]:
import os
HOME = os.getcwd()
print(HOME)

/content


Extraemos la carpeta Oracle.zip en este caso en /tmp/Oracle. En esta carpeta se encuentran las imagenes de entrenamiento, validacion y test, con la extructura necesaria para que yolo interprete las diferentes clases de los datos. Esta estructura es la siguiente:

- Oracle
  - train
    - 0
    - 1
    - 2
    - 3
    - 4
    - 5
    - 6
    - 7
  - val
    - 0
    - 1
    - 2
    - 3
    - 4
    - 5
    - 6
    - 7
  - test
    - 0
    - 1
    - 2
    - 3
    - 4
    - 5
    - 6
    - 7

In [4]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/oracle/Oracle.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/tmp/Oracle') #Extracts the files into the /tmp folder
zip_ref.close()

Instalamos ultralytics, la libreria de yolov8 que vamos a utilizar para entrenar nuestro modelo.

In [5]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.0/78.2 GB disk)


Procedemos a entrenar el modelo. Vamos a utilizar de base el modelo "yolov8l-cls.pt", pero puede ser cualquier otro modelo. El parametro "data" contiene la ruta a los datos que van a ser utilizados en el entrenamiento con la estrucutra mencionada anteriormente. Tambien definimos el numero de epoch que queramos utilizar, un batch size (dependiendo de la tarjeta grafica asignada podra ser mayor o menor), activamos la flag de cache si nos lo permite el sistema (para que no tenga que cargar las imagenes cada vez que las necesite), y el modelo a utilizar como base.

El resto de hiperparametros no los cambiamos, ya que los valores por defecto funcionan muy bien y las pruebas que hemos realizado modificandolos no han mejorado el resultado.

Cabe mencionar que por ejemplo se realiza "data augmentation" haciendo uso de la popular libreria albumentations.

Una vez iniciado el entrenamiento podemos acceder a la url que nos proporciona ngrok para monitorizar el proceso de entrenamiento.

In [25]:
from ultralytics import YOLO

model = YOLO("yolov8l-cls.pt")

# Train the model for classifying food
model.train(data = "/tmp/Oracle/data", epochs= 100, batch=80, cache = True, model = "yolov8l-cls.pt")

Ultralytics YOLOv8.0.54 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=classify, mode=train, model=yolov8x-cls.pt, data=/tmp/Oracle/data, epochs=100, patience=50, batch=80, imgsz=224, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=Fa

Realizamos clasificacion con el modelo obtenido de los datos de test, y guardamos el resultado en un json con el formato que nos pide el concurso.

In [26]:
clasification = model("/tmp/Oracle/test")

import pandas as pd
import json
test_csv = pd.read_csv("/content/drive/MyDrive/oracle/test.csv")



dic = {"target":{}}

for i in range(len(clasification)):
    label = clasification[i].probs.argmax().item()
    name = clasification[i].path.split("/")[-1]
    indice = test_csv.loc[test_csv['path_img'].str.contains(name), 'idx_test'].values[0]
    dic["target"][str(indice)] = int(label)


with open('submission.json', 'w') as f:
    f.write(json.dumps(dic, indent=2))

  0%|          | 0.00/755k [00:00<?, ?B/s]

image 1/4797 /tmp/Oracle/test/0005574f-e1fd-4262-bdc2-fc8a99819796.jpeg: 224x224 0 1.00, 6 0.00, 4 0.00, 3 0.00, 1 0.00, 23.8ms
image 2/4797 /tmp/Oracle/test/000c3b4f-7a49-49cd-b7ac-4b4136fe747e.jpeg: 224x224 7 0.99, 2 0.00, 6 0.00, 1 0.00, 3 0.00, 24.5ms
image 3/4797 /tmp/Oracle/test/001376b0-6c0f-44bb-b27a-538c7ee409a2.jpeg: 224x224 1 0.96, 0 0.02, 7 0.02, 6 0.00, 5 0.00, 24.0ms
image 4/4797 /tmp/Oracle/test/001551b7-3062-4564-ae2c-3bdc7a94be35.jpeg: 224x224 7 0.47, 3 0.24, 5 0.14, 4 0.14, 6 0.00, 23.8ms
image 5/4797 /tmp/Oracle/test/0020a9b8-0396-49cc-8126-b15f549f296d.jpeg: 224x224 4 1.00, 6 0.00, 3 0.00, 0 0.00, 5 0.00, 23.7ms
image 6/4797 /tmp/Oracle/test/0026892f-68c3-4416-99cd-90e9df8f246f.jpeg: 224x224 3 0.83, 0 0.09, 1 0.04, 4 0.02, 5 0.01, 23.8ms
image 7/4797 /tmp/Oracle/test/002ccf89-e45f-45de-8da2-fdb4abf8d745.jpeg: 224x224 2 1.00, 4 0.00, 6 0.00, 5 0.00, 0 0.00, 23.8ms
image 8/4797 /tmp/Oracle/test/00414198-d487-41fd-b823-8c242e474fc1.jpeg: 224x224 3 0.96, 1 0.03, 4 0.00,

(Opcional) Si queremos descargar los modelos entrenados asi como la diferente información guardada del entrenamiento, ejecutamos esta celda.

In [ ]:
import zipfile
import os

folder_name = '/content/runs'
zip_name = folder_name + '.zip'

# Crear archivo ZIP vacío
with zipfile.ZipFile(zip_name, mode='w') as zipf:
    pass

# Agregar contenido de la carpeta al archivo ZIP
with zipfile.ZipFile(zip_name, mode='a') as zipf:
    for root, dirs, files in os.walk(folder_name):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path)

print(f'Carpeta "{folder_name}" comprimida en el archivo "{zip_name}"')


from google.colab import files
files.download("/content/runs.zip")


KeyboardInterrupt: ignored